In [ ]:
import os
from pathlib import Path

import numpy as np

from ho_stray_field import *
from ho_stray_field.gs import GS, superpotential_factors, superpotential
from ho_stray_field.bspline import BSpline
from ho_stray_field.tensor_grid import TensorGrid
from ho_stray_field.sources import flower_state

jax.config.update("jax_enable_x64", True)

# Fitting the super-potential

Here we compare the superpotential, computed at some sample points with adaptive Gauss-Konrod, with the separable approximation. We use a fixed rank $r=40$ and vary the order of the B-spline ansatz which fitts a flower state.

First we load the precomputed result of the superpotential. It was computed with adaptive Gauss-Konrod on a $10\times10\times10$ equidistant grid.

In [2]:
with open(Path(os.path.abspath("")).parent / "test" / "functional" / "sp_flower.npy", "rb") as f:
    sp_flower = np.load(f)

And next we define the function which computes the error of the separable approximation. The superpotential is directly evaluated and no further fitting is used.

In [3]:
def compute_sp_err(k, r=40):
    # first we fit the flower state with the B-spline model
    tg_m = TensorGrid(*([jnp.linspace(-0.5, 0.5, r)] * 3))
    elm_m = BSpline(tg_m, degree=k-1)
    tg_quad = TensorGrid(*([jnp.linspace(-0.5, 0.5, 2)] * 3)).to_gauss(150)
    mag = elm_m.fit(tg_quad, flower_state)

    # then we compute the superpotential factor matrices
    gs = GS.from_sinc_1_over_sqrtx(100, 1.9)
    _t = (jnp.linspace(-0.5, 0.5, sp_flower.shape[i]) for i in range(3))
    _targets = TensorGrid(*_t)
    sp_factors, _ = superpotential_factors(elm_m, _targets, tg_m, gs,
                                              epsabs=1e-13, epsrel=0.0, order=31, max_ninter=200)

    # and evaluate if with the core of the magnetization
    gs_result = superpotential(mag.core, sp_factors, gs)
    err = jnp.max(jnp.abs(gs_result - sp_flower))
    return err


for k in [2, 3, 4, 5, 6]:
    err = compute_sp_err(k, 40)
    print(f"k={k}, r=40, err={err}")


k=2, r=40, err=9.448206439444107e-08
k=3, r=40, err=1.0116585347219598e-10
k=4, r=40, err=7.760458942129844e-14
k=5, r=40, err=2.930988785010413e-14
k=6, r=40, err=2.6867397195928788e-14
